# Fase 2: Definição do Alvo e Engenharia de Atributos (EAP 2.0)

Este notebook implementa a Fase 2 do projeto de previsão de tendência do IBOVESPA, focando na criação da variável a ser prevista e no enriquecimento dos dados com atributos preditivos, com atenção rigorosa para evitar viés de lookahead.

**Autor:** Projeto Tech Challenge 2  
**Data:** 2025-01-24  
**Referência:** EAP.md e Steering.md

## ⚠️ Compatibilidade NumPy 2.0+

**Problema identificado**: O `pandas_ta` não é compatível com NumPy 2.0+ devido à remoção do `numpy.NaN`.

**Solução**: Implementamos os indicadores técnicos manualmente usando apenas pandas e numpy, garantindo compatibilidade total.

## 1. Importações e Configurações

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette('husl')

# Verifica versões para compatibilidade
print('=== VERIFICAÇÃO DE COMPATIBILIDADE ===')
print(f'✓ Pandas: {pd.__version__}')
print(f'✓ NumPy: {np.__version__}')
print('✓ Bibliotecas importadas com sucesso!')
print('✓ Usando implementação manual para indicadores técnicos')

## 2. Funções Auxiliares (Compatíveis com NumPy 2.0+)

Implementamos manualmente os indicadores técnicos:

In [ ]:
def calcular_sma(serie, periodo):
    """
    Calcula a Média Móvel Simples (SMA)
    Compatível com NumPy 2.0+
    """
    return serie.rolling(window=periodo, min_periods=periodo).mean()

def calcular_retornos_log(serie):
    """
    Calcula retornos logarítmicos
    Compatível com NumPy 2.0+
    """
    return np.log(serie / serie.shift(1))

def criar_lags_retornos(retornos, max_lag=5):
    """
    Cria lags dos retornos
    Compatível com NumPy 2.0+
    """
    lags = {}
    for lag in range(1, max_lag + 1):
        lags[f'Return_Lag_{lag}'] = retornos.shift(lag)
    return lags

print('✓ Funções auxiliares definidas com sucesso!')
print('✓ Implementação manual dos indicadores técnicos')
print('✓ Totalmente compatível com NumPy 2.0+')

## 3. Carregamento dos Dados

In [ ]:
# Carrega os dados limpos da Fase 1
dados_limpos = pd.read_csv('dados_bovespa.csv', index_col=0, parse_dates=True)

print(f'✓ Dados carregados: {len(dados_limpos)} registros')
print(f'✓ Período: {dados_limpos.index.min()} a {dados_limpos.index.max()}')
print(f'✓ Colunas: {list(dados_limpos.columns)}')

# Visualiza as primeiras linhas
print('\n=== PRIMEIRAS LINHAS DOS DADOS ===')
dados_limpos.head()

## 4. Criação da Variável Alvo (Target)

Criamos a variável alvo seguindo as especificações do EAP 2.0:
- **Target = 1** se Close(t+1) > Close(t)
- **Target = 0** caso contrário

In [ ]:
# Copia os dados para não modificar o original
dados_com_target = dados_limpos.copy()

# Identifica coluna de fechamento
col_close = None
for col in dados_com_target.columns:
    if 'close' in col.lower():
        col_close = col
        break

if col_close is None:
    raise ValueError('Coluna de preço de fechamento não encontrada')

print(f'✓ Coluna de fechamento identificada: {col_close}')

# Cria a variável alvo
close_amanha = dados_com_target[col_close].shift(-1)
close_hoje = dados_com_target[col_close]
dados_com_target['Target'] = (close_amanha > close_hoje).astype(int)

# Remove a última linha que contém NaN no Target
dados_com_target = dados_com_target.dropna(subset=['Target'])

print('\n=== CRIAÇÃO DA VARIÁVEL ALVO ===')
print('✓ Variável Target criada com sucesso')
print('✓ Lógica: Target = 1 se Close(t+1) > Close(t), 0 caso contrário')
print(f'✓ Registros após remoção de NaN: {len(dados_com_target)}')

# Visualiza os primeiros registros com o target
print('\n=== PRIMEIROS REGISTROS COM TARGET ===')
dados_com_target[['Open', 'High', 'Low', 'Close', 'Target']].head(10)

## 5. Análise da Distribuição de Classes

In [ ]:
# Calcula e analisa a frequência das classes 0 e 1
distribuicao = dados_com_target['Target'].value_counts()
proporcoes = dados_com_target['Target'].value_counts(normalize=True)

# Análise de desbalanceamento
classe_majoritaria = proporcoes.idxmax()
prop_majoritaria = proporcoes.max()
prop_minoritaria = proporcoes.min()
razao_desbalanceamento = prop_majoritaria / prop_minoritaria

print('=== ANÁLISE DA DISTRIBUIÇÃO DE CLASSES ===')
print('✓ Distribuição absoluta:')
for classe, count in distribuicao.items():
    print(f'   - Classe {classe}: {count} ({proporcoes[classe]:.2%})')

print('\n✓ Análise de desbalanceamento:')
print(f'   - Classe majoritária: {classe_majoritaria} ({prop_majoritaria:.2%})')
print(f'   - Razão de desbalanceamento: {razao_desbalanceamento:.2f}:1')

if razao_desbalanceamento > 1.5:
    print('   - Status: DESBALANCEADO (razão > 1.5)')
    print('   - Implicações:')
    print('     • Acurácia não é uma métrica confiável')
    print('     • Usar Precisão, Recall e F1-Score')
    print('     • Considerar scale_pos_weight no XGBoost')
else:
    print('   - Status: BALANCEADO (razão <= 1.5)')

# Visualização gráfica
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico de barras
distribuicao.plot(kind='bar', ax=ax1, color=['red', 'green'])
ax1.set_title('Distribuição Absoluta das Classes')
ax1.set_xlabel('Classe Target')
ax1.set_ylabel('Frequência')
ax1.tick_params(axis='x', rotation=0)

# Gráfico de pizza
proporcoes.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=['red', 'green'])
ax2.set_title('Distribuição Percentual das Classes')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

## 6. Engenharia de Atributos

### 6.1 Atributos de Momento (Lags de Retornos)

In [ ]:
# Copia os dados com target para adicionar features
dados_com_features = dados_com_target.copy()

# Calcula retornos logarítmicos usando função manual
dados_com_features['Retornos_Log'] = calcular_retornos_log(dados_com_features[col_close])

# Dicionário para armazenar informações dos atributos criados
dicionario_atributos = {}

# Cria lags dos retornos (1 a 5 dias)
print('=== CRIANDO ATRIBUTOS DE MOMENTO ===')
print('✓ Usando implementação manual (compatível com NumPy 2.0+)')

lags_retornos = criar_lags_retornos(dados_com_features['Retornos_Log'], 5)
for col_name, serie_lag in lags_retornos.items():
    dados_com_features[col_name] = serie_lag
    lag_num = col_name.split('_')[-1]
    
    # Adiciona ao dicionário de atributos
    dicionario_atributos[col_name] = {
        'calculo': f'log(Close(t-{lag_num}) / Close(t-{lag_num}-1))',
        'dados_entrada': 'Close',
        'intuicao_financeira': f'Captura o momentum de {lag_num} dia(s) atrás',
        'categoria': 'Momento'
    }
    print(f'✓ Criado: {col_name}')

print('\n✓ Atributos de momento criados com sucesso!')
print('✓ Capturam informações de momentum de curto prazo')

# Visualiza os novos atributos
colunas_momento = [f'Return_Lag_{i}' for i in range(1, 6)]
print('\n=== PRIMEIROS REGISTROS COM ATRIBUTOS DE MOMENTO ===')
dados_com_features[['Close', 'Retornos_Log'] + colunas_momento + ['Target']].head(10)

### 6.2 Atributos de Tendência (SMAs e Ratios)

In [ ]:
# Cria médias móveis simples usando função manual
periodos_sma = [5, 10, 20]

print('=== CRIANDO ATRIBUTOS DE TENDÊNCIA ===')
print('✓ Usando implementação manual (compatível com NumPy 2.0+)')

for periodo in periodos_sma:
    col_name = f'SMA_{periodo}'
    dados_com_features[col_name] = calcular_sma(dados_com_features[col_close], periodo)
    
    # Adiciona ao dicionário de atributos
    dicionario_atributos[col_name] = {
        'calculo': f'Média móvel simples de {periodo} períodos',
        'dados_entrada': 'Close',
        'intuicao_financeira': f'Tendência de {"curto" if periodo <= 10 else "médio"} prazo',
        'categoria': 'Tendência'
    }
    print(f'✓ Criado: {col_name}')

# Cria ratios entre preço e SMAs
print('\n=== CRIANDO RATIOS PREÇO/SMA ===')
for periodo in periodos_sma:
    col_name = f'Close_SMA_{periodo}_Ratio'
    dados_com_features[col_name] = dados_com_features[col_close] / dados_com_features[f'SMA_{periodo}']
    
    # Adiciona ao dicionário de atributos
    dicionario_atributos[col_name] = {
        'calculo': f'Close / SMA_{periodo}',
        'dados_entrada': 'Close, SMA',
        'intuicao_financeira': f'Posição relativa do preço vs tendência de {periodo} dias',
        'categoria': 'Tendência'
    }
    print(f'✓ Criado: {col_name}')

# Cria ratios entre SMAs
print('\n=== CRIANDO RATIOS ENTRE SMAS ===')
dados_com_features['SMA_5_10_Ratio'] = dados_com_features['SMA_5'] / dados_com_features['SMA_10']
dados_com_features['SMA_10_20_Ratio'] = dados_com_features['SMA_10'] / dados_com_features['SMA_20']

# Adiciona ao dicionário
dicionario_atributos['SMA_5_10_Ratio'] = {
    'calculo': 'SMA_5 / SMA_10',
    'dados_entrada': 'SMA_5, SMA_10',
    'intuicao_financeira': 'Divergência entre tendências de curto e médio prazo',
    'categoria': 'Tendência'
}

dicionario_atributos['SMA_10_20_Ratio'] = {
    'calculo': 'SMA_10 / SMA_20',
    'dados_entrada': 'SMA_10, SMA_20',
    'intuicao_financeira': 'Divergência entre tendências de médio prazo',
    'categoria': 'Tendência'
}

print('✓ Criado: SMA_5_10_Ratio')
print('✓ Criado: SMA_10_20_Ratio')

print('\n✓ Atributos de tendência criados com sucesso!')
print('✓ Capturam informações sobre direção e força das tendências')

## 7. Finalização e Limpeza dos Dados

In [ ]:
# Remove registros com NaN (causados pelos lags e médias móveis)
dados_finais = dados_com_features.dropna()

print('=== FINALIZAÇÃO DOS DADOS ===')
print(f'✓ Registros antes da limpeza: {len(dados_com_features)}')
print(f'✓ Registros após limpeza: {len(dados_finais)}')
print(f'✓ Registros removidos: {len(dados_com_features) - len(dados_finais)}')
print(f'✓ Total de colunas: {len(dados_finais.columns)}')

# Lista todas as colunas criadas
colunas_originais = dados_limpos.columns.tolist()
colunas_criadas = [col for col in dados_finais.columns if col not in colunas_originais]

print(f'\n✓ Colunas criadas ({len(colunas_criadas)}): {colunas_criadas}')

# Verifica distribuição final do target
distribuicao_final = dados_finais['Target'].value_counts(normalize=True)
print(f'\n✓ Distribuição final do Target:')
for classe, prop in distribuicao_final.items():
    print(f'   - Classe {classe}: {prop:.2%}')

# Salva os dados processados
dados_finais.to_csv('dados_fase2_completos.csv')
print(f'\n✓ Dados salvos em: dados_fase2_completos.csv')

# Salva o dicionário de atributos
with open('dicionario_atributos_fase2.json', 'w', encoding='utf-8') as f:
    json.dump(dicionario_atributos, f, indent=2, ensure_ascii=False)
print(f'✓ Dicionário salvo em: dicionario_atributos_fase2.json')

# Visualiza o dataset final
print('\n=== DATASET FINAL ===')
dados_finais.head()

## 8. Resumo da Fase 2

### ✅ Objetivos Alcançados:

1. **✅ Compatibilidade NumPy 2.0+**: Resolvido problema com pandas_ta
2. **✅ Criação da Variável Alvo**: Target binário baseado na direção do preço
3. **✅ Análise de Balanceamento**: Verificação da distribuição das classes
4. **✅ Atributos de Momento**: 5 lags de retornos para capturar momentum
5. **✅ Atributos de Tendência**: SMAs e ratios para capturar tendências
6. **✅ Documentação Completa**: Dicionário detalhado de todos os atributos
7. **✅ Prevenção de Lookahead**: Implementada corretamente

### 🎯 Próximos Passos:

1. **Fase 3**: Preparação para modelagem (normalização, divisão treino/teste)
2. **Fase 4**: Treinamento e validação de modelos
3. **Fase 5**: Análise de resultados e conclusões

### 💡 Solução de Compatibilidade:

- **Problema**: pandas_ta incompatível com NumPy 2.0+
- **Solução**: Implementação manual de indicadores técnicos
- **Resultado**: Código totalmente compatível e controlado